<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Notion - Get database
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Notion/Notion_Get_database.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

**Tags:** #notion #productivity

## Input

### Import library

In [1]:
from naas_drivers import notion

### Variables
<a href='https://docs.naas.ai/drivers/notion'>How to get your Notion integration token ?</a>

In [9]:
# Enter Token API
token = "*****"

# Enter Database URL
database_url = "https://www.notion.so/********"

## Model

In [10]:
database = notion.connect(token).database.get(database_url)

## Output

In [11]:
database

,Reviewer,Type,People,Platform,Assign,Link,Media,Publication Date,Views,Tags,Status,Name,Likes,Sender,Comments
0,,,,,,None,,,None,,Published ✨,,None,,None
1,,,,LinkedIn,,None,,2021-10-10,None,,Backlog 💡,🗓⚡️ 66DAYS #10,None,,None
2,,,,,,None,,,None,,Backlog 💡,66DAYS,None,,None
3,,,,LinkedIn,Jeremy Ravenel,None,,2021-10-21,None,,Backlog 💡,🗓⚡️ 66DAYS #21,None,,None
4,,,,LinkedIn,,None,,2021-10-09,None,,Backlog 💡,🗓⚡️ 66DAYS #9 - How to get my stats on Github?,None,,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,,Educational,,LinkedIn,Jeremy Ravenel,None,,2021-10-12,None,,Backlog 💡,How to send automatically your LinkedIn stats ...,None,Jeremy Ravenel,None
96,,,,LinkedIn,Valentin Piquard,None,,2021-06-23,None,,Published ✨,AFTE - Data mining pour les trésoriers,None,Jeremy Ravenel,None
97,,Educational,,LinkedIn,Valentin Piquard,None,,2021-10-18,None,,Backlog 💡,🌎📩 Create yourself a daily brief of the world ...,None,Jeremy Ravenel,None
98,,Entertaining,,LinkedIn,Florent Ravenel,None,,2021-10-08,None,,Backlog 💡,📊 Naas makes your Power BI report go faster,None,Jeremy Ravenel,None
